In [1]:
import warnings
warnings.filterwarnings("ignore")

from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features
from helper.fairness_functions import statistical_measures, print_conf_matrix, print_statistical_measures, calc_fairness_metrics, get_metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'


In [2]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

In [3]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
columns_to_exclude = ['age', 'ability to speak english', 'gave birth this year']
# Encoding the features and target, and excluding some columns
X_encoded, y_encoded = encode_all_features(X, y, [])
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [4]:
X_train.head()

,age,education,workinghours,ability to speak english,marital status_Divorced,marital status_Husband,marital status_Never married,marital status_Separated,marital status_Widowed,marital status_Wife,...,occupation_Office/Administrative Support,occupation_Production/Assembly,occupation_Protective Services,occupation_Repair/Maintenance,occupation_Sales,"occupation_Science, Engineering, Technology",occupation_Service/Hospitality,occupation_Transport,sex_Female,sex_Male
6317,22,16,36,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
740,61,22,40,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3781,48,16,40,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
7850,62,18,65,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2963,53,19,44,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [5]:
X_test['sex'] = X_test['sex_Male'] * 1
X_train['sex'] = X_train['sex_Male'] * 1
X_test = X_test.drop(columns=['sex_Male', 'sex_Female'])
X_train = X_train.drop(columns=['sex_Male', 'sex_Female'])

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
DI, DS, EO, EOdds, conf_matrix = statistical_measures(X_test, y_test, y_pred, 'sex', use_lib_implementation=False)
    
print_statistical_measures(DI, DS, EO, EOdds)

print_conf_matrix(conf_matrix)

Disparate Impact (DI): 0.525
Discrimination Score (DS): -0.198
Equal Opportunity Difference (EO): 0.179
Equalized Odds (EOdds): 0.097

Confusion Matrix: 
Percentage of True positives = 0.21777777777777776
Percentage of True negatives = 0.5177777777777778
Percentage of False positives = 0.135
Percentage of False negatives = 0.12944444444444445
FPR:  0.20680851063829786
TPR:  0.6272
PPP:  0.6173228346456693


In [6]:
def split_male_female_metrics(model, X_train, X_test, y_train, y_test):

    X_male_train = X_train[X_train['sex_Male'] == 1]
    X_male_test = X_test[X_test['sex_Male'] == 1]
    y_male_train = y_train[X_train['sex_Male'] == 1]
    y_male_test = y_test[X_test['sex_Male'] == 1]

    y_male_pred = model.predict(X_male_test)

    male_accuracy, male_precision, male_recall, male_f1 = get_metrics(y_male_test, y_male_pred)

    X_female_train = X_train[X_train['sex_Male'] == 0]
    X_female_test = X_test[X_test['sex_Male'] == 0]
    y_female_train = y_train[X_train['sex_Male'] == 0]
    y_female_test = y_test[X_test['sex_Male'] == 0]

    y_female_pred = model.predict(X_female_test)

    female_accuracy, female_precision, female_recall, female_f1 = get_metrics(y_female_test, y_female_pred)

    # calculate male conf matrix
    conf_matrix_male = confusion_matrix(y_male_test, y_male_pred)
    # calculate male conf matrix
    conf_matrix_female = confusion_matrix(y_female_test, y_female_pred)

    print("\nConfusion Matrix for Male group")
    print_conf_matrix(conf_matrix_male)
    print("\nConfusion Matrix for Female group")
    print_conf_matrix(conf_matrix_female)

    # Compare the model's accuracy for male and female groups
    metrics_table = pd.DataFrame({
        'Gender': ['Male', 'Female'],
        'Accuracy': [male_accuracy, female_accuracy],
        'Precision': [male_precision, female_precision],
        'Recall': [male_recall, female_recall],
        'F1-score': [male_f1, female_f1]
    })

    return metrics_table

In [7]:
model = DecisionTreeClassifier(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
metrics_table = split_male_female_metrics(model, X_train, X_test, y_train, y_test)
metrics_table


Confusion Matrix for Male group

Confusion Matrix: 
Percentage of True positives = 0.24917218543046357
Percentage of True negatives = 0.4478476821192053
Percentage of False positives = 0.1390728476821192
Percentage of False negatives = 0.16390728476821192
FPR:  0.23695345557122707
TPR:  0.6032064128256514
PPP:  0.6417910447761194

Confusion Matrix for Female group

Confusion Matrix: 
Percentage of True positives = 0.10304054054054054
Percentage of True negatives = 0.6554054054054054
Percentage of False positives = 0.13175675675675674
Percentage of False negatives = 0.1097972972972973
FPR:  0.16738197424892703
TPR:  0.48412698412698413
PPP:  0.43884892086330934


,Gender,Accuracy,Precision,Recall,F1-score
0,Male,0.697020,0.641791,0.603206,0.621901
1,Female,0.758446,0.438849,0.484127,0.460377


### Resampling the dataset

In [8]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=42)

X_, y_ = get_features_and_target(data, 'income')
X_, y_ = encode_all_features(X_, y_, ['sex'])
X_['sex'] =  X_['sex'].map({'Female': 0, 'Male': 1})

X_with_y = pd.concat([X_, y_], axis=1)

X_ = X_with_y.drop(columns=['sex'])
y_ = X_with_y['sex']

# Resample the dataset
X_resampled, y_resampled = adasyn.fit_resample(X_, y_)

X_with_y_resampled = pd.concat([X_resampled, y_resampled], axis=1)

In [9]:
X_with_y_resampled['sex'].value_counts() / len(X_with_y_resampled) * 100

sex
1    52.232959
0    47.767041
Name: count, dtype: float64

In [10]:
counts = X_with_y_resampled.groupby(['sex', 'income']).size().unstack(fill_value=0)
total_counts = X_with_y_resampled['sex'].value_counts()
# Calculate the percentage of each income level for each sex
percentage = counts.div(total_counts, axis=0) * 100

In [11]:
percentage

income,0,1
sex,,
0,84.344815,15.655185
1,58.900000,41.100000


In [12]:
X_, y_ = get_features_and_target(X_with_y_resampled, 'income')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
DI, DS, EO, EOdds, conf_matrix = statistical_measures(X_test, y_test, y_pred, 'sex', use_lib_implementation=False)
    
print_statistical_measures(DI, DS, EO, EOdds)

print_conf_matrix(conf_matrix)

Disparate Impact (DI): 0.355
Discrimination Score (DS): -0.252
Equal Opportunity Difference (EO): 0.094
Equalized Odds (EOdds): 0.161

Confusion Matrix: 
Percentage of True positives = 0.17275892080069627
Percentage of True negatives = 0.6100957354221062
Percentage of False positives = 0.10008703220191471
Percentage of False negatives = 0.11705831157528286
FPR:  0.1409313725490196
TPR:  0.5960960960960962
PPP:  0.6331738437001595


In [14]:
X_with_y_resampled['sex'] =  X_with_y_resampled['sex'].map({0: 'Female', 1: 'Male'})
X_with_y_resampled = pd.get_dummies(X_with_y_resampled, columns=['sex'], dtype=int)
model = DecisionTreeClassifier(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_with_y_resampled.drop(columns=['income']), X_with_y_resampled['income'], test_size=0.2, random_state=42)
model.fit(X_train, y_train)
metrics_table = split_male_female_metrics(model, X_train, X_test, y_train, y_test)


Confusion Matrix for Male group

Confusion Matrix: 
Percentage of True positives = 0.2534750613246116
Percentage of True negatives = 0.4309076042518397
Percentage of False positives = 0.15699100572363042
Percentage of False negatives = 0.15862632869991825
FPR:  0.2670375521557719
TPR:  0.6150793650793651
PPP:  0.6175298804780877

Confusion Matrix for Female group

Confusion Matrix: 
Percentage of True positives = 0.0827906976744186
Percentage of True negatives = 0.7637209302325582
Percentage of False positives = 0.08558139534883721
Percentage of False negatives = 0.06790697674418604
FPR:  0.10076670317634173
TPR:  0.5493827160493827
PPP:  0.49171270718232046


In [15]:
metrics_table

,Gender,Accuracy,Precision,Recall,F1-score
0,Male,0.684383,0.617530,0.615079,0.616302
1,Female,0.846512,0.491713,0.549383,0.518950
